# spm1d for IVD testing in AO

## Libraries and constants
#### Notes
Mech 2 "Injured" samples exclude 4 and 13

In [ ]:
import os
import re
import spm1d
import time
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import multiprocessing as mp

from scipy.signal import savgol_filter
from tqdm import tqdm

print("libs imported")

In [ ]:
# General Constants
datatypes = ['Mod1 [MPa]','Mod2 [MPa]','L1 [frac]','L2 [frac]','E [J]']
path = %pwd
timepoints = ['Mech0','Mech1','Mech2','Mech3','Mech4']
timepoints_human = {'Mech0':'After 1 Day','Mech1':'After 2 Day','Mech2':'After 7 Days','Mech2':'After 8 Days','Mech2':'After 14 Days',}
file = 'DataforSPMRAW_new.xlsx'

print('constants found; path: ' + path)

# State  and Sample Constatns
data_samples = {'Mech0':[8,8,8],'Mech1':[8,8,8],'Mech2':[6,8,8],'Mech3':[8,8,8],'Mech4':[8,8,8]}
sample_sheet = {'Mech0':['statehorz1','statehorz2','statehorz3'],
                'Mech1':['statehorz1','statehorz2','statehorz3'],
                'Mech2':['statehorz1','statehorz2','statehorz3'],
                'Mech3':['statehorz1','statehorz2','statehorz3'],
                'Mech4':['statehorz1','statehorz2','statehorz3']}
state_name = {'Mech0':['Intact','Repaired', 'Injured'],
              'Mech1':['Intact','Repaired', 'Injured'],
              'Mech2':['Intact','Repaired', 'Injured'],
              'Mech3':['Intact','Repaired', 'Injured'],
              'Mech4':['Intact','Repaired', 'Injured']}

print('constants found; ' + time.ctime())

## Import and shape data

In [ ]:
#for i in timepoints:
#    print(i)
#    sample = []
#    for k in state_name[i]:
#       print(state_name[i]) 

# Import data from excel
data = []
y = 0
while y < len(timepoints):
    sample = []
    x = 0
    while x < len(state_name[timepoints[y]]):
        sample.append(pd.read_excel(path + '/' + timepoints[y] + '/' + file,sheet_name=sample_sheet[timepoints[y]][x]))
        x += 1 
    data.append(sample)    
    y += 1

print("import done")

# Shape data into dictionary
o = 0
z = 0
samplenum = str(0)
spmsamples = []
spmsamples_s_t_d = {}
while z < len(datatypes):
    datatype = datatypes[z]
    y = 0
    spmsamples_s_t = {}
    while y < len(timepoints):
        x = 0
        spmsamples_s = {}
        while x < len(state_name[timepoints[y]]):
            spmsample = []
            u = 1
            while u < (data_samples[timepoints[y]][x] + 1):
                # get data from imported lists
                if u == 1:
                    spmsample.append(data[y][x][datatype])
                else:
                    samplenum = str(u - 1)
                    spmsample.append(data[y][x][datatype + '.' + samplenum])
                u += 1
                o += 1
                
                # print to console
#                print(str(o) + ': '
#                      + datatype + '; '
#                      + timepoints[y] + '; '
#                      + 'state ' + state_name[timepoints[y]][x] 
#                      + ' (' + sample_sheet[timepoints[y]][x] + '); '
#                      + 'sample ' + str(u-1) + '/' + str(data_samples[timepoints[y]][x])
#                      + '; (y=' + str(y) + ', x=' + str(x) 
#                      +  ', datatype=' + datatype + '.' + samplenum + ')'
#                      + '\n')
            # remove NaNs and replace with 0
            spmarrsample = []
            spmarrsample = np.array(spmsample)
            spmarrsample[np.isnan(spmarrsample)] = 0
            
            
            # write to data matrix
            spmsamples_s[state_name[timepoints[y]][x]] = spmarrsample
            x += 1
        spmsamples_s_t[timepoints[y]] = spmsamples_s
        y += 1
    #dictionary with array of all states s:{1,2,3}, 
    #timepoints t:{Mec1,...Mech4}, measurements d:{Mod1,...L1}
    spmsamples_s_t_d[datatypes[z]] = spmsamples_s_t 
    z += 1
    
print('shaping done')
        

## Calculations

In [ ]:
spmsamples_s_t_d_calc = spmsamples_s_t_d 
for timepoint in timepoints:
    print
    if timepoint == 'Mech0':
        lastTimepoint = 'Mech0'
    else: 
        for datatype in datatypes:
            for state in state_name[timepoint]:
                print('Working on ' + timepoint + ', ' + datatype + ', ' + state)
                after = spmsamples_s_t_d[datatype][timepoint][state]
                before = spmsamples_s_t_d[datatype][lastTimepoint][state]
                spmsamples_s_t_d_calc[datatype][timepoint][state] = ( after - before ) / before * 100
        lastTimepoint = timepoint

## Plot

### Plot Averages

In [ ]:
# sample colours
colours = ['mediumseagreen','indianred','royalblue']
# create x-axis
datarange = list(range(0,180*5,5))
# define axis limits
axis_ylim = {'Mod1 [MPa]':(-40,80),
            'Mod2 [MPa]':(-20,60),
            'L1 [frac]':(-30,60),
            'L2 [frac]':(-30,60),
            'E [J]':(-30,60)}
# Plot average graphs

z = 0
while z < len(datatypes):
    datatype = datatypes[z]
    y = 0
    while y < len(timepoints):
        x = 0
        timepoint = timepoints[y]
        fig, ax = plt.subplots() # initialise figure

        while x < len(state_name[timepoints[y]]):
            state = state_name[timepoints[y]][x]
            
            # get data
            plotdata = []            
            plotdata = spmsamples_s_t_d[datatype][timepoint][state_name[timepoints[y]][x]]
            
            # plot
            plt.plot(datarange, plotdata.mean(axis=0), color=colours[x],lw=3, label=state)
            
            minstd = np.add(plotdata.mean(axis=0),plotdata.std(axis=0))
            maxstd = np.subtract(plotdata.mean(axis=0),plotdata.std(axis=0))
            
            plt.plot(datarange, minstd, color=colours[x], lw=1, alpha=0.3)
            plt.plot(datarange, maxstd, color=colours[x], lw=1, alpha=0.3)
            plt.fill_between(datarange, minstd, maxstd, alpha=0.2, facecolor=colours[x] )

            #plt.plot(datarange, plotdata.max(axis=0), color=colours[x], lw=2, ls='--')
            #plt.plot(datarange, plotdata.min(axis=0), color=colours[x], lw=2, ls='--')
            
            ax.set_ylim(axis_ylim[datatype])
            plt.title(timepoints_human[timepoint], fontsize=15)
            plt.xlabel('Time [sec]', size=15)
            plt.ylabel(datatype + ' Change [%]', size=15)
            plt.legend(loc='upper right', fontsize=12)

            x += 1
        
        #save figure
        figurename = datatype + ' ' + timepoint
        plt.savefig(path + '/Average Figures/' + figurename + '.png', format='png', bbox_inches='tight')
        plt.savefig(path + '/Average Figures/' + figurename + '.svg', format='svg', bbox_inches='tight')
        plt.close()
        
        y += 1
    z += 1
print('plotting done')

### Plot individual graphs

In [ ]:
# create x-axis
datarange = list(range(0,180*5,5))
# define axis limits
axis_ylim = {'Mod1 [MPa]':(-40,100),
            'Mod2 [MPa]':(-20,60),
            'L1 [frac]':(-30,60),
            'L2 [frac]':(-30,60),
            'E [J]':(-30,60)}
# Plot average graphs

z = 0
while z < len(datatypes):
    datatype = datatypes[z]
    y = 0
    while y < len(timepoints):
        x = 0
        timepoint = timepoints[y]
        while x < len(state_name[timepoints[y]]):
            state = state_name[timepoints[y]][x]
            # get data
            plotdata = []            
            plotdata = spmsamples_s_t_d[datatype][timepoint][state_name[timepoints[y]][x]]
            # initialise figure
            fig, ax = plt.subplots()
            g = 0
            while g < len(plotdata):
                plt.plot(savgol_filter(plotdata[g],21,5))
                g += 1
            figurename = timepoint + ' ' + datatype + ' ' + state
            plt.title(figurename, fontsize=15)
            ax.set_ylim(axis_ylim[datatype])    
            plt.xlabel('Cycle', size=15)
            plt.ylabel(datatype, size=15)
            
            plt.savefig(path + '/Individual Figures/' + 'Ind ' + figurename + '.png', format='png', bbox_inches='tight')
            plt.savefig(path + '/Individual Figures/' + 'Ind ' + figurename + '.svg', format='svg', bbox_inches='tight')
            plt.close()
            
            x += 1
        y += 1
    z += 1
print('plotting done')

## Stats

In [ ]:
var1 = 'Mod1 [MPa]'
var2 = 'Mod2 [MPa]'
timepoint = 'Mech1'
state1 = 'Injured'
state2 = 'Intact'

len(spmsamples_s_t_d[var1][timepoint][state1])

In [ ]:
# multi-dimensional SPM
var1 = 'Mod1 [MPa]'
var2 = 'Mod2 [MPa]'
timepoint = 'Mech1'
state1 = 'Injured'
state2 = 'Intact'

spmsamples_s_t_d[var1][timepoint][state1],spmsamples_s_t_d[var2][timepoint][state2]


#(1) Conduct test:
alpha        = 0.05
T2           = spm1d.stats.hotellings_paired(YA, YB)
T2i          = T2.inference(0.05)
print(T2i)


#(2) Plot:
plt.close('all')
ax0     = plt.subplot(221)
ax1     = plt.subplot(222)
ax3     = plt.subplot(224)
## plot SPM results:
ax0.plot(YA[:,:,0].T, 'k', label='slow')
ax0.plot(YB[:,:,0].T, 'r', label='fast')
ax1.plot(YA[:,:,1].T, 'k')
ax1.plot(YB[:,:,1].T, 'r')
T2i.plot(ax=ax3)
plt.show()

In [ ]:
# Individual SPM Plots

var = 'Mod1 [MPa]'
timepoint = 'Mech1'
state1 = 'Delaminated'
state2 = 'Intact'


# t-test
alpha = 0.05
F  = spm1d.stats.nonparam.ttest2(spmsamples_s_t_d[var][timepoint][state1],spmsamples_s_t_d[var][timepoint][state2])
Fi = F.inference(alpha=0.05, two_tailed=True,iterations=70)
Fi.plot()
Fi.plot_threshold_label(bbox=dict(facecolor='w'))
#axes = plt.gca()
#axes.set_ylim([0,5])
plt.title(timepoints_human[timepoint] + ' ' + state1 + ' vs. ' + state2, fontsize=15)
plt.xlabel('Cycle', size=20 )
figurename = var + ' ' + timepoint  + ' ' + state1 + ' vs. ' + state2
plt.savefig(path + '/Stats/' + figurename + '.png', format='png', bbox_inches='tight')
plt.savefig(path + '/Stats/' + figurename + '.svg', format='svg', bbox_inches='tight')

In [ ]:
# Non parametric t-test
SPMData = pd.DataFrame(columns = ['timepoint','datatype','state1','state2','Sig','p','F','Fi'])
SPMData = SPMData[0:0]
# Loop
n=0
for timepoint in state_name:
    for datatype in datatypes:
        for state in list(itertools.combinations(state_name[timepoint],2)):
            F  = spm1d.stats.nonparam.ttest2(spmsamples_s_t_d[datatype][timepoint][state[0]],spmsamples_s_t_d[datatype][timepoint][state[1]])
            Fi = F.inference(alpha=0.05,two_tailed=False,iterations=70)
            sig = Fi.h0reject
            p = Fi.p
            if sig == True:
                print(timepoint+' '+datatype+' '+state[1]+'-'+state[0]+': p = '+str(p))
            SPMData.loc[n] = [timepoint,datatype,state[1],state[0],sig,p,F,Fi]
            n+=1
print('calculations done')